In [1]:
# Importing libraries

import numpy as np
import time 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd


from science_jubilee.Machine import Machine
from science_jubilee.tools.Tool import Tool
from science_jubilee.decks.Deck import Deck
from science_jubilee.labware.Labware import Labware, Location, Well
from science_jubilee.tools.Pipette import Pipette
from science_jubilee.tools.Double_Syringe import DoubleSyringe
from science_jubilee.tools.Vacuum_Gripper import VacuumGripper
from science_jubilee.tools.Oceandirect_axo import Spectrometer

In [ ]:
# Manual Offsetting 

deck = Deck("lab_automation_deck") # We are not using the deck, so operate any deck.json 

# -------------Position JSON file for Pipette ---------------------#
tiprack = deck.load_labware(labware_filename = 'opentrons_96_tiprack_1000ul_axo.json', slot = 5)   # Load an opentrons 20uL
tiprack.manual_offset2([[274.1, 219.8],[173.1, 220.0],[173.1, 282.0]], save = True)   # Save through list of list. List of coordinates should be A1 A12 H12 subsequently. 
samples_ot2 = deck.load_labware("uwsunlab_12_wellplate_4000ul_pipette_1.json", slot = 2)
samples_ot2.manual_offset2([[131.3, 135.5], [36.5, 135.8], [36.5, 176.8]], save = True)
samples2_ot2 = deck.load_labware("uwsunlab_12_wellplate_4000ul_pipette_2.json", slot = 0)
samples2_ot2.manual_offset2([[129.8, 40], [35.9, 38.8], [35.9, 79.8]], save = True)
solvent = deck.load_labware("uwsunlab_1_reservoir_60000ul.json", slot = 3)
solvent.manual_offset_small_plate([[225.2, 155.8]], save = True)
    
# -------------Position JSON file for Dual Syringe ---------------#
samples_sy = deck.load_labware('uwsunlab_12_wellplate_4000ul_1.json', slot = 2)
samples_sy.manual_offset2([[132.7, 132.6],[38.6, 132.6], [38.6, 174.2]], save = True)
samples2_sy = deck.load_labware('uwsunlab_12_wellplate_4000ul_2.json', slot = 0)
samples2_sy.manual_offset2([[132.2, 36.3], [38.2, 36.3], [38.2, 77.5]], save = True)
precursors = deck.load_labware('uwsunlab_2_wellplate_60000ul.json', slot = 1)
precursors.manual_offset_small_plate([[245.0, 60.0], [197.0, 60.0]], save = True)
# -----------Position JSON file for Spectrometer----------------# 
samples_spec = deck.load_labware('uwsunlab_12_wellplate_4000ul_spec_1.json', slot = 2)
samples_spec.manual_offset2([[128.3, 132.8],[34.0, 132.8], [34.0, 173.8]], save = True)
samples2_spec = deck.load_labware('uwsunlab_12_wellplate_4000ul_spec_2.json', slot = 0)
samples2_spec.manual_offset2([[127.3, 35.6], [32.9, 35.6], [33.5, 77.4]], save = True)

 

# ----------Position JSON file for Vacuum Gripper---------------#

Manual offset applied to opentrons_96_tiprack_1000ul
Manual offset saved
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset saved
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset saved
Manual offset applied (1×1 plate),  x_adjust=0.0, y_adjust=0.0
Manual offset saved.
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset saved
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset saved
Manual offset applied (1×2 plate),  x_adjust=0.0, y_adjust=0.0
Manual offset saved.
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset saved
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset saved


In [ ]:
# Intialising and homing the Machine 

axo = Machine(address = '192.168.1.2')

In [ ]:
axo.home_all()
axo.move_to(z=180) # Safest Z location

In [5]:
# Loading the labwares

deck = axo.load_deck("lab_automation_deck")

## ---------------------OT2 1000ul Pipette ----------------------------##
tiprack = axo.load_labware("opentrons_96_tiprack_1000ul_axo.json", 5)
tiprack.load_manualOffset()
samples_ot2 = axo.load_labware('uwsunlab_12_wellplate_4000ul_pipette_1.json', 2)
samples_ot2.load_manualOffset()
samples2_ot2 = axo.load_labware('uwsunlab_12_wellplate_4000ul_pipette_2.json', 0)
samples2_ot2.load_manualOffset()
solvent = axo.load_labware('uwsunlab_1_reservoir_60000ul.json', 3)
solvent.load_manualOffset()
trash = (-0.9, 55.8, 99)
## -------------------------------------------------------------------##
#pipette = Pipette(0, "Pipette", brand = "opentron", model = "config", max_volume= 20, min_volume= 0, zero_position = (0,0,0),\
#    blowout_position=None, drop_tip_position = None, mm_to_ul = None)# model = "config")

## ---------------------Dual Syringe Pump ----------------------------##
samples_sy = axo.load_labware('uwsunlab_12_wellplate_4000ul_1.json', 2)
samples_sy.load_manualOffset()
samples2_sy = axo.load_labware('uwsunlab_12_wellplate_4000ul_2.json', 0)
samples2_sy.load_manualOffset()
precursors = axo.load_labware('uwsunlab_2_wellplate_60000ul.json', 1)
precursors.load_manualOffset()
## ------------------------------------------------------------------##

## --------------------------Spectrometer -------------------------------##
samples_spec = axo.load_labware('uwsunlab_12_wellplate_4000ul_spec_1.json', 2)
samples_spec.load_manualOffset()
samples2_spec = axo.load_labware('uwsunlab_12_wellplate_4000ul_spec_2.json', 0)
samples2_spec.load_manualOffset()
## ----------------------------------------------------------------------##

## ------------------------Vacuum Gripper --------------------------------##
vacuum_location = [(84, 51, 0),  # Slot 0 (12 Well Plate)
                   (225, 57, 0), # Slot 1 (2 Precursor Container)
                   (84, 147, 0), # Slot 2 (12 Well Plate)
                   (225, 149, 0), # Slot 3 (1 Solvent Container)
                   (79, 244, 0)] # Slot 4 (Lid Containter)



Manual offset applied to opentrons_96_tiprack_1000ul
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset applied (1×1 plate),  x_adjust=0.0, y_adjust=0.0
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset applied (1×2 plate),  x_adjust=0.0, y_adjust=0.0
Manual offset applied to uwsunlab_12_wellplate_4000ul
Manual offset applied to uwsunlab_12_wellplate_4000ul


In [5]:
# Load the spectrometer

spectrometer = Spectrometer(index = 3, 
                            name = 'Spectrometer', 
                            base_dir=r"C:\Users\ADITI\Downloads\Aditya\Axo_Jubilee\science-jubilee\axo\spectrum_data",
                            plate_id = 'testing_4',
                            ref_dark = 'dark_20250528_144107.npy',
                            ref_white = 'white_20250528_144214.npy')
# axo.load_tool(spectrometer)

RuntimeError: No Ocean Insight Spectrometers Detected

In [63]:
axo.pickup_tool(spectrometer)

In [64]:
spectrometer.configure_device()

In [65]:
spectrometer.set_dark() 

'dark_20250625_160225'

In [66]:
spectrometer.set_white()

'white_20250625_160230'

In [60]:
spectrometer.collect_spectrum(samples_spec[0].top(-5), 15, save= True)

(array([ 194.02957153,  194.57849831,  195.1273039 , ..., 1102.96248167,
        1103.3101147 , 1103.657674  ], shape=(2068,)),
 array([-128.12509754, -157.23825152, -136.35669405, ..., -211.85013739,
        -238.55423364, -238.21959584], shape=(2068,)))

In [ ]:
# Position Probe and take readings one by one. 
# Also plot the Absorbance spectrum side by side. 

for i in range(6):
    # spectrometer.position_probe(samples_spec[i].top(-5))
    spectrometer.collect_spectrum(samples_spec[i].top(-5), elapsed_min=15, save= True)

    # Single spectrum plot
    wavelengths, absorbance = spectrometer.plot_spectrum(samples_spec[0], elasped_min=15, show_plot=True, save_plot=True)    

In [61]:
axo.park_tool()

In [ ]:
# Data Processing and Visualisation 


# # Paths to your files
# csv_path = r"C:\Users\ADITI\Downloads\Aditya\Axo_Jubilee\science-jubilee\axo\spectrum_data\testing_3\A6.csv"
# dark_path = r"C:\Users\ADITI\Downloads\Aditya\Axo_Jubilee\science-jubilee\axo\spectrum_data\refs\dark_20250528_144107.npy"
# white_path = r"C:\Users\ADITI\Downloads\Aditya\Axo_Jubilee\science-jubilee\axo\spectrum_data\refs\white_20250528_144214.npy"

# # Load dark and white reference values
# dark_vals = np.load(dark_path)
# white_vals = np.load(white_path)

# # Read the sample CSV, skipping metadata lines
# sample_df = pd.read_csv(csv_path, comment='#')

# # Extract wavelength and intensity
# wavelengths = sample_df.iloc[:, 0].values
# sample_vals = sample_df.iloc[:, 1].values

# # --- Compute Absorbance ---
# absorbance = spectrometer.compute_absorbance(sample_vals, dark_vals, white_vals)

# # --- Plot ---
# plt.figure(figsize=(10, 6))
# plt.plot(wavelengths, absorbance, color='purple', label='Absorbance')
# plt.xlabel("Wavelength (nm)")
# plt.ylabel("Absorbance (AU)")
# plt.title("Wavelength vs Absorbance")
# plt.grid(True)
# plt.legend()
# plt.tight_layout()
# plt.show()


NameError: name 'spectrometer' is not defined